In [1]:
library("xgboost")
library("Matrix")
library('Ckmeans.1d.dp')
library('lightgbm')



Attaching package: ‘lightgbm’


The following object is masked from ‘package:xgboost’:

    slice




## Read data and process data labels

In [2]:
time_matrix <- matrix(0,ncol = 3, nrow =4)
colnames(time_matrix) <- c("user_time", "system_time", "elapsed_time")
start_time = Sys.time()

In [3]:
data=read.csv('Daily_features_matrix.csv')
data=data[,2:dim(data)[2]]

In [4]:
dim(data)

[1] 4227   23

In [5]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,⋯,diff1_acf10,diff2_acf1,diff2_acf10,length,user_time,system_time,elapsed_time,user_time.1,system_time.1,elapsed_time.1
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0,1,0.9977307,2.180566e-11,31.265883,4.202931,0.8853252,3.0007563,0.009502084,⋯,0.007626758,-0.5188252,0.2734535,1006,0.02104115,0.02104115,0.02104115,0,0,0
2,1,0,1,0.9753638,2.434650e-09,24.625465,-1.039634,0.8973350,3.0475729,0.211225819,⋯,0.004072618,-0.5203490,0.2801685,1006,0.02027750,0.02027750,0.02027750,0,0,0
3,1,0,1,0.8937784,8.333451e-06,3.804175,4.843726,0.4029370,0.3356327,0.610410200,⋯,0.002531199,-0.4987115,0.2487157,130,0.01750016,0.01750016,0.01750016,0,0,0
4,1,0,1,0.7424974,8.486237e-06,1.762132,6.219671,0.4733951,0.4720488,0.740639635,⋯,0.058828311,-0.5022730,0.3038757,169,0.02131033,0.02131033,0.02131033,0,0,0
5,1,0,1,0.9764692,1.782445e-07,11.344076,0.859238,0.6058021,0.8285215,0.281090710,⋯,0.122022465,-0.4771979,0.3813929,156,0.15410614,0.15410614,0.15410614,0,0,0
6,1,0,1,0.9963228,8.446153e-11,29.701579,10.112177,0.8651846,3.0647427,0.043039068,⋯,0.076311068,-0.5852963,0.4167008,1006,0.01933837,0.01933837,0.01933837,0,0,0


In [6]:
dlist= load('Daily_ets_datalist.RData')
datalist=eval(parse(text = dlist ))
res=datalist[[1]]
MASE=res[,,,6]
m=5

In [7]:
dim(MASE)

[1] 4227    5    4

In [8]:
whichmin<-function(x){
    minx=min(x[x>0])
    loc=which(x==minx)[1]-1
    loc
}

meanunique=function(x)
    {
    mean(unique(x))
}

In [9]:
nanum=c()
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    a=apply(count,1,min)
    if(sum(is.na(a))>0)
        {
        nanum=append(nanum,i)
    }
    }

In [10]:
nanum

NULL

In [11]:
realbestmin=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    min_value=apply(count,1,min)
    if (max(min_value,na.rm = TRUE)==0){
        realbestmin[i,]=0
        }
    else
        {
        min_value[is.na(min_value)]=100 
        realbestmin[i,]= whichmin(min_value)
    }
    }

In [12]:
table(realbestmin)

realbestmin
   0    1    2    3    4 
 763  733  332  675 1724 

In [13]:
realbestmean=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    mean_value=apply(count,1,meanunique)
    if (max(mean_value,na.rm = TRUE)==0){
        realbestmean[i,]=0
        }
    else
        {
        mean_value[is.na(mean_value)]=100 
        realbestmean[i,]= whichmin(mean_value)
    }
    }

In [14]:
realbestmean

3
4
4
3
0
3
4
2
2
1
0


In [15]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,⋯,diff1_acf10,diff2_acf1,diff2_acf10,length,user_time,system_time,elapsed_time,user_time.1,system_time.1,elapsed_time.1
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0,1,0.9977307,2.180566e-11,31.265883,4.202931,0.8853252,3.0007563,0.009502084,⋯,0.007626758,-0.5188252,0.2734535,1006,0.02104115,0.02104115,0.02104115,0,0,0
2,1,0,1,0.9753638,2.434650e-09,24.625465,-1.039634,0.8973350,3.0475729,0.211225819,⋯,0.004072618,-0.5203490,0.2801685,1006,0.02027750,0.02027750,0.02027750,0,0,0
3,1,0,1,0.8937784,8.333451e-06,3.804175,4.843726,0.4029370,0.3356327,0.610410200,⋯,0.002531199,-0.4987115,0.2487157,130,0.01750016,0.01750016,0.01750016,0,0,0
4,1,0,1,0.7424974,8.486237e-06,1.762132,6.219671,0.4733951,0.4720488,0.740639635,⋯,0.058828311,-0.5022730,0.3038757,169,0.02131033,0.02131033,0.02131033,0,0,0
5,1,0,1,0.9764692,1.782445e-07,11.344076,0.859238,0.6058021,0.8285215,0.281090710,⋯,0.122022465,-0.4771979,0.3813929,156,0.15410614,0.15410614,0.15410614,0,0,0
6,1,0,1,0.9963228,8.446153e-11,29.701579,10.112177,0.8651846,3.0647427,0.043039068,⋯,0.076311068,-0.5852963,0.4167008,1006,0.01933837,0.01933837,0.01933837,0,0,0


In [16]:
set.seed(100)
index = sample(2,nrow(data),replace = TRUE,prob=c(0.7,0.3))

In [17]:
train_data=data[index==1,]
test_data=data[index==2,]
train_label_min=realbestmin[index==1,]
test_label_min=realbestmin[index==2,]
train_label_mean=realbestmean[index==1,]
test_label_mean=realbestmean[index==2,]

In [18]:
head(train_data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,⋯,diff1_acf10,diff2_acf1,diff2_acf10,length,user_time,system_time,elapsed_time,user_time.1,system_time.1,elapsed_time.1
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0,1,0.9977307,2.180566e-11,31.265883,4.202931,0.8853252,3.0007563,0.009502084,⋯,0.007626758,-0.5188252,0.2734535,1006,0.02104115,0.02104115,0.02104115,0,0,0
2,1,0,1,0.9753638,2.434650e-09,24.625465,-1.039634,0.8973350,3.0475729,0.211225819,⋯,0.004072618,-0.5203490,0.2801685,1006,0.02027750,0.02027750,0.02027750,0,0,0
3,1,0,1,0.8937784,8.333451e-06,3.804175,4.843726,0.4029370,0.3356327,0.610410200,⋯,0.002531199,-0.4987115,0.2487157,130,0.01750016,0.01750016,0.01750016,0,0,0
4,1,0,1,0.7424974,8.486237e-06,1.762132,6.219671,0.4733951,0.4720488,0.740639635,⋯,0.058828311,-0.5022730,0.3038757,169,0.02131033,0.02131033,0.02131033,0,0,0
5,1,0,1,0.9764692,1.782445e-07,11.344076,0.859238,0.6058021,0.8285215,0.281090710,⋯,0.122022465,-0.4771979,0.3813929,156,0.15410614,0.15410614,0.15410614,0,0,0
6,1,0,1,0.9963228,8.446153e-11,29.701579,10.112177,0.8651846,3.0647427,0.043039068,⋯,0.076311068,-0.5852963,0.4167008,1006,0.01933837,0.01933837,0.01933837,0,0,0


In [19]:
end_time = Sys.time()

In [20]:
time_matrix[1,]=end_time-start_time

In [21]:
end_time-start_time

Time difference of 6.007752 secs

## Target the interval where the actual error is minimum

In [22]:
start_time = Sys.time()

In [23]:
dtrain_xg_min_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_min)) 
dtrain_xg_min_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)) )

In [24]:
dtrain_lg_min_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_min))
dtrain_lg_min_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)))

In [25]:
xgb_min_reg <- xgboost(data = dtrain_xg_min_reg, nround=100)

[1]	train-rmse:1.968889 
[2]	train-rmse:1.606489 
[3]	train-rmse:1.385961 
[4]	train-rmse:1.245172 
[5]	train-rmse:1.154625 
[6]	train-rmse:1.088893 
[7]	train-rmse:1.048818 
[8]	train-rmse:1.026792 
[9]	train-rmse:1.001343 
[10]	train-rmse:0.985571 
[11]	train-rmse:0.964205 
[12]	train-rmse:0.940648 
[13]	train-rmse:0.915263 
[14]	train-rmse:0.894765 
[15]	train-rmse:0.880601 
[16]	train-rmse:0.872627 
[17]	train-rmse:0.867789 
[18]	train-rmse:0.846039 
[19]	train-rmse:0.826617 
[20]	train-rmse:0.823262 
[21]	train-rmse:0.817698 
[22]	train-rmse:0.810733 
[23]	train-rmse:0.795340 
[24]	train-rmse:0.783779 
[25]	train-rmse:0.771382 
[26]	train-rmse:0.763028 
[27]	train-rmse:0.753728 
[28]	train-rmse:0.751335 
[29]	train-rmse:0.745779 
[30]	train-rmse:0.739230 
[31]	train-rmse:0.729188 
[32]	train-rmse:0.724897 
[33]	train-rmse:0.709020 
[34]	train-rmse:0.701841 
[35]	train-rmse:0.691101 
[36]	train-rmse:0.685054 
[37]	train-rmse:0.678692 
[38]	train-rmse:0.673345 
[39]	train-rmse:0.671

In [26]:
xgb_min_cl <- xgboost(data = dtrain_xg_min_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.369160 
[2]	train-mlogloss:1.224522 
[3]	train-mlogloss:1.116148 
[4]	train-mlogloss:1.031653 
[5]	train-mlogloss:0.966375 
[6]	train-mlogloss:0.913142 
[7]	train-mlogloss:0.865078 
[8]	train-mlogloss:0.825468 
[9]	train-mlogloss:0.794339 
[10]	train-mlogloss:0.763620 
[11]	train-mlogloss:0.735455 
[12]	train-mlogloss:0.711015 
[13]	train-mlogloss:0.692737 
[14]	train-mlogloss:0.673113 
[15]	train-mlogloss:0.649362 
[16]	train-mlogloss:0.632835 
[17]	train-mlogloss:0.618217 
[18]	train-mlogloss:0.604652 
[19]	train-mlogloss:0.588697 
[20]	train-mlogloss:0.571914 
[21]	train-mlogloss:0.552500 
[22]	train-mlogloss:0.538981 
[23]	train-mlogloss:0.524607 
[24]	train-mlogloss:0.514780 
[25]	train-mlogloss:0.503443 
[26]	train-mlogloss:0.493192 
[27]	train-mlogloss:0.482516 
[28]	train-mlogloss:0.470824 
[29]	train-mlogloss:0.459367 
[30]	train-mlogloss:0.446797 
[31]	train-mlogloss:0.436461 
[32]	train-mlogloss:0.425232 
[33]	train-mlogloss:0.415421 
[34]	train-mlogloss

In [27]:
lgb_min_reg <- lgb.train(data = dtrain_lg_min_reg, nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014713 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4321
[LightGBM] [Info] Number of data points in the train set: 2968, number of used features: 17
[LightGBM] [Info] Start training from score 2.477089


In [28]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_min_cl <- lgb.train(data = dtrain_lg_min_cl,nrounds = 100,params=params)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027395 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4321
[LightGBM] [Info] Number of data points in the train set: 2968, number of used features: 17
[LightGBM] [Info] Start training from score -1.734152
[LightGBM] [Info] Start training from score -1.801238
[LightGBM] [Info] Start training from score -2.519180
[LightGBM] [Info] Start training from score -1.830226
[LightGBM] [Info] Start training from score -0.873584


In [29]:
end_time = Sys.time()
time_matrix[2,]=end_time-start_time

## Target the interval where the average error is minimum

In [22]:
start_time = Sys.time()

In [23]:
dtrain_xg_mean_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_mean)) 
dtrain_xg_mean_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)) )

In [24]:
dtrain_lg_mean_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_mean))
dtrain_lg_mean_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)))

In [25]:
xgb_mean_reg <- xgboost(data = dtrain_xg_mean_reg, nround=100)

[1]	train-rmse:1.798705 
[2]	train-rmse:1.491756 
[3]	train-rmse:1.297863 
[4]	train-rmse:1.182610 
[5]	train-rmse:1.098489 
[6]	train-rmse:1.044787 
[7]	train-rmse:1.009266 
[8]	train-rmse:0.970277 
[9]	train-rmse:0.953962 
[10]	train-rmse:0.938634 
[11]	train-rmse:0.927464 
[12]	train-rmse:0.902347 
[13]	train-rmse:0.894438 
[14]	train-rmse:0.884128 
[15]	train-rmse:0.859286 
[16]	train-rmse:0.847007 
[17]	train-rmse:0.836520 
[18]	train-rmse:0.818882 
[19]	train-rmse:0.803682 
[20]	train-rmse:0.797958 
[21]	train-rmse:0.792398 
[22]	train-rmse:0.774690 
[23]	train-rmse:0.756420 
[24]	train-rmse:0.742690 
[25]	train-rmse:0.729922 
[26]	train-rmse:0.721511 
[27]	train-rmse:0.716746 
[28]	train-rmse:0.703808 
[29]	train-rmse:0.700756 
[30]	train-rmse:0.693675 
[31]	train-rmse:0.677594 
[32]	train-rmse:0.670546 
[33]	train-rmse:0.661254 
[34]	train-rmse:0.652034 
[35]	train-rmse:0.639474 
[36]	train-rmse:0.636191 
[37]	train-rmse:0.631693 
[38]	train-rmse:0.617920 
[39]	train-rmse:0.613

In [26]:
xgb_mean_cl <- xgboost(data = dtrain_xg_mean_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.389089 
[2]	train-mlogloss:1.251838 
[3]	train-mlogloss:1.148491 
[4]	train-mlogloss:1.068132 
[5]	train-mlogloss:1.007891 
[6]	train-mlogloss:0.952153 
[7]	train-mlogloss:0.901189 
[8]	train-mlogloss:0.859373 
[9]	train-mlogloss:0.815601 
[10]	train-mlogloss:0.788992 
[11]	train-mlogloss:0.759369 
[12]	train-mlogloss:0.733064 
[13]	train-mlogloss:0.708714 
[14]	train-mlogloss:0.688706 
[15]	train-mlogloss:0.670262 
[16]	train-mlogloss:0.651944 
[17]	train-mlogloss:0.635615 
[18]	train-mlogloss:0.616026 
[19]	train-mlogloss:0.598535 
[20]	train-mlogloss:0.584770 
[21]	train-mlogloss:0.571014 
[22]	train-mlogloss:0.554973 
[23]	train-mlogloss:0.540874 
[24]	train-mlogloss:0.528446 
[25]	train-mlogloss:0.519715 
[26]	train-mlogloss:0.507161 
[27]	train-mlogloss:0.492352 
[28]	train-mlogloss:0.479348 
[29]	train-mlogloss:0.468325 
[30]	train-mlogloss:0.459523 
[31]	train-mlogloss:0.445907 
[32]	train-mlogloss:0.434674 
[33]	train-mlogloss:0.419950 
[34]	train-mlogloss

In [27]:
lgb_mean_reg <- lgb.train(data = dtrain_lg_mean_reg,nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.303048 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4321
[LightGBM] [Info] Number of data points in the train set: 2968, number of used features: 17
[LightGBM] [Info] Start training from score 2.176550


In [ ]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_mean_cl <- lgb.train(data = dtrain_lg_mean_cl,params=params,nrounds = 100)

In [ ]:
end_time = Sys.time()
time_matrix[3,]=end_time-start_time

In [ ]:
end_time-start_time

## predict

In [39]:
start_time = Sys.time()

In [40]:
alldatalgb <- lgb.Dataset(data = as.matrix(data))
alldataxgb <- xgb.DMatrix(data = as.matrix(data))


In [41]:
xgbregmin=predict(xgb_min_reg,alldataxgb)
xgbclsmin=predict(xgb_min_cl,alldataxgb)
lgbregmin=predict(lgb_min_reg,as.matrix(data))
lgbclsmin=predict(lgb_min_cl,as.matrix(data))

In [42]:
xgbregmin

[1]  3.9025821686  3.7359621525  3.8243637085  3.8958027363  2.0674850941
   [6]  3.9609832764  2.3272991180  2.8803529739  2.4772956371  1.9050450325
  [11]  1.3893365860  1.5588865280  1.3723167181  3.9656915665  2.3234293461
  [16]  0.2529275715  0.2971372008  0.6890478730  3.9447770119  0.0401622504
  [21]  4.0728230476  3.3822982311  3.9769232273  4.2826666832  4.1070899963
  [26]  3.9969556332  4.0044169426  2.3214747906  3.1575791836  3.7763946056
  [31]  0.1468557417 -0.1875828058  0.2985766530  2.3015334606  2.0361785889
  [36]  1.6920251846  2.7640390396  4.0737090111  3.0747034550  3.1869382858
  [41]  3.9414336681  3.1250615120  2.3991160393  1.2758606672  0.9502615333
  [46]  1.7076199055  1.9058748484  2.7290310860  3.0804405212  3.8870277405
  [51]  4.0899848938  1.5219845772  0.8260059953  3.8722095490  0.1306731701
  [56]  3.1332859993  3.3820040226  3.6129972935  3.7209067345  2.9684278965
  [61]  1.0026162863  3.6647758484  0.9660137296  0.2575529218  2.9031748772
  [66]  3.7155761719  4.0035753250  2.2299575806 -0.0125877764  3.8530466557
  [71]  1.4842963219  3.7682709694  3.8798043728  1.1986997128  2.6780929565
  [76]  4.0890865326  3.4751048088  1.7714992762  3.5225260258  3.0601820946
  [81]  3.9682974815  3.7308225632  2.4843120575  1.3303550482  1.3993092775
  [86]  3.7560811043  0.6229026318  0.0110628158 -0.0259916373  0.5768898129
  [91]  0.8677653074  0.0640689880 -0.1102855727 -0.0046795914  0.4508991241
  [96] -0.1326138079 -0.0441613868  3.7863571644  3.2380878925  2.2162563801
 [101]  3.6230416298  3.4409177303  3.2528665066  3.7938690186  3.7817320824
 [106]  0.2871263921  0.1786890626  3.3011806011  3.6076934338  3.7881262302
 [111]  1.6905399561  2.2946305275  3.8316257000  2.9933164120  2.4126439095
 [116]  2.7432208061  1.0080419779 -0.5247769356 -0.0086149452  0.7422169447
 [121]  0.3683571219  0.5109227300  2.6914727688  2.7229955196  0.0714979321
 [126]  1.8306838274  1.8671768904  2.1634323597  4.0643796921  1.7858663797
 [131]  2.9055559635  2.4758896828  2.8033270836  0.8364186287  3.1106314659
 [136]  3.1907582283  3.9462060928  3.5231912136  0.1392023712  2.0196862221
 [141]  3.9952328205  3.9696881771  3.8685123920  0.7082148790  3.5114672184
 [146]  0.9969770908  2.1185426712  3.9691541195  3.1055700779  1.7055902481
 [151]  2.2512762547  1.6343793869  1.9530030489  3.7395555973  3.6029982567
 [156]  4.0915269852  3.5436632633  1.7408109903  3.8700156212  1.9778947830
 [161]  2.4750652313  4.0720729828  1.1253650188  0.9652759433  0.9259986877
 [166]  2.9978396893  3.9730024338  2.1288945675  2.4900896549  0.3378100693
 [171]  0.2403726876  0.5493963957  0.6187412739  1.5504121780  2.8536539078
 [176]  3.7834951878  4.1178746223  3.6824162006  0.0268513933  0.1579593569
 [181]  0.1282038391 -0.5717951059  1.7436420918  2.9897150993  3.1789669991
 [186]  2.4855434895  3.7147955894  3.1209232807  3.2339181900  2.9221184254
 [191]  4.0900549889  2.7721834183  1.3639786243  3.3747997284  3.7551670074
 [196]  3.8295772076  0.3876462281  1.2696989775  1.2049015760  0.5682045221
 [201]  3.9504148960  2.8887808323  2.7883622646  2.4894392490  1.0713009834
 [206]  2.1140632629  3.3768656254  3.9350073338  4.0227580070  3.8927135468
 [211]  4.0027732849  2.0257148743  3.9314420223  0.4957470894  3.8288660049
 [216]  3.5979516506  0.0872822776  1.8757925034  3.4361653328  3.5935022831
 [221]  0.0030073598  2.0222423077  3.8631618023  2.2854747772  3.8529174328
 [226]  0.0011242571 -0.0332970843  0.0334096402  0.8039231896  2.1149964333
 [231]  3.6888482571  3.6662828922  3.1205704212  0.2147607952  2.6913280487
 [236]  2.4761228561  2.1281747818  2.7331216335  2.2190437317  1.6335867643
 [241]  0.7137323618  3.7730615139  2.1064949036  3.6338322163  3.7222802639
 [246]  3.0121805668  3.0157201290  2.1351413727  0.1993612647  0.0146913659
 [251]  0.2499008924  1.3345876932  1.8335565329  1.4097679853  1.1274716854
 [256]  1.3585146666  1.7367184162  3.8941910267  1.6906547546  1.2908213139

In [43]:
xgbclsmin

[1] 4 4 4 4 2 4 4 3 3 2 1 4 1 4 4 0 0 0 4 0 4 4 4 4 4 4 4 3 3 4 0 0 0 4 2 0 3
  [38] 4 4 3 4 0 4 4 0 2 3 4 3 4 4 1 0 4 0 3 4 4 4 3 1 4 0 0 3 4 4 2 0 4 1 4 4 1
  [75] 3 4 4 1 3 3 4 4 0 0 1 4 0 0 0 0 0 0 0 0 0 0 0 4 3 0 4 4 4 4 4 0 0 4 4 4 4
 [112] 4 4 3 2 3 4 0 0 0 0 0 3 3 0 2 2 4 4 2 3 4 4 0 3 3 4 4 0 2 4 4 4 4 4 1 1 4
 [149] 3 2 2 1 2 4 4 4 4 0 4 2 2 4 1 1 1 3 4 4 2 0 0 0 0 4 4 4 3 4 0 0 0 0 4 3 3
 [186] 2 4 3 3 4 4 3 1 4 4 4 0 1 0 4 4 4 4 0 1 2 4 4 4 4 4 4 4 0 4 4 1 4 4 4 0 2
 [223] 4 4 4 0 0 0 1 2 4 4 0 0 2 2 2 4 4 0 0 4 2 4 4 4 3 2 0 0 0 1 2 1 1 4 1 4 1
 [260] 4 2 4 0 1 0 0 0 4 0 4 4 4 3 1 1 4 1 4 4 4 4 4 4 2 4 4 4 4 4 0 4 0 0 2 0 0
 [297] 0 0 4 0 4 2 4 1 1 0 4 4 4 2 1 4 2 4 4 4 4 0 0 0 0 0 0 0 0 1 2 2 2 2 2 1 1
 [334] 1 1 2 2 3 2 3 3 4 3 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 0 4 4 4 4 4 1 0 0 0 0 2
 [371] 0 0 0 0 0 4 4 4 3 0 0 0 0 1 4 1 4 4 0 4 1 4 4 1 1 4 4 3 0 4 4 4 4 4 4 4 3
 [408] 4 3 3 3 4 4 4 0 0 0 0 1 4 3 4 4 4 4 4 1 0 2 2 4 0 2 3 4 4 4 4 1 2 4 4 3 4
 [445] 4 0 4 3 4 0 0 4 4 4 3 4 4 3 4 4 4 4 4 1 4 1 4 4 4 4 1 3 4 1 4 4 4 4 4 4 4
 [482] 1 4 4 1 1 1 1 1 4 0 1 4 0 4 0 1 4 1 4 1 0 4 0 3 4 4 1 0 3 3 0 0 0 0 0 0 0
 [519] 1 0 0 0 1 1 1 1 0 4 1 1 2 3 1 4 3 4 3 1 4 3 1 2 2 2 0 4 1 1 0 4 1 1 0 1 3
 [556] 4 0 0 4 0 4 4 0 4 4 0 4 4 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 4 3 4 0 0 3 3
 [593] 4 4 3 0 3 3 4 4 4 3 3 4 3 3 3 3 3 0 3 0 4 0 0 0 0 0 0 0 0 0 0 0 0 4 0 3 2
 [630] 0 0 2 2 2 4 4 4 4 0 3 2 2 0 3 2 2 2 4 4 0 2 2 2 2 3 3 2 2 3 0 3 1 0 3 3 3
 [667] 3 3 3 3 3 3 4 2 4 1 3 3 4 4 4 4 0 3 4 2 3 4 4 4 4 4 3 4 4 3 1 4 4 4 0 3 3
 [704] 4 2 4 4 2 4 4 4 4 2 4 4 4 4 3 3 4 3 3 3 3 4 3 4 4 3 4 3 4 2 3 3 3 3 4 3 1
 [741] 1 1 4 0 0 1 1 1 4 0 0 1 1 1 0 1 0 1 1 0 1 1 1 1 4 0 1 1 4 3 3 4 1 2 3 4 4
 [778] 3 4 1 4 1 0 1 3 0 2 4 3 4 3 4 0 4 0 4 3 0 1 1 2 4 4 4 0 4 4 4 3 4 4 0 4 4
 [815] 4 4 4 4 4 4 4 4 4 1 1 3 4 3 3 1 3 3 4 3 1 3 4 4 4 4 4 4 1 0 2 3 3 3 4 3 0
 [852] 3 0 3 3 0 4 3 3 3 3 3 4 0 3 0 1 1 1 0 1 3 3 1 3 1 1 1 1 1 1 1 1 1 1 2 1 1
 [889] 1 4 1 1 4 0 1 1 1 2 1 1 1 1 4 1 0 1 1 1 1 4 1 1 4 4 4 4 4 4 2 0 4 4 4 4 4
 [926] 4 4 4 4 2 3 4 3 4 4 4 4 4 4 4 4 4 1 1 1 1 1 1 1 1 1 4 0 0 0 0 0 0 0 4 0 0
 [963] 0 0 0 0 0 0 0 1 0 3 0 4 0 0 1 0 1 4 1 0 3 1 1 4 4 4 4 4 4 4 4 4 3 4 1 4 4
[1000] 4 4 3 1 4 0 4 4 1 3 4 4 4 0 4 4 3 4 4 4 4 0 4 4 4 1 0 0 0 4 0 0 2 0 0 0 0
[1037] 3 0 1 1 1 1 1 0 1 1 0 0 1 1 4 2 2 0 2 1 1 1 4 2 1 4 3 0 3 1 1 0 2 4 4 3 4
[1074] 2 4 4 4 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 4 0 4 4 4 3 4 2 3 3 4 4 4 2 4
[1111] 1 0 4 3 3 4 4 4 4 3 4 4 2 4 3 4 4 0 2 4 1 1 2 1 1 1 4 1 0 1 2 1 0 0 1 2 2
[1148] 2 1 2 0 1 0 1 1 0 0 0 2 0 0 2 0 0 0 0 1 0 0 0 0 0 1 0 1 0 4 1 1 1 1 0 1 1
[1185] 1 1 4 4 4 3 0 4 1 1 0 4 4 0 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 3 0 4 3 2 4 4 4
[1222] 3 4 2 4 4 4 4 4 3 0 0 0 0 0 1 1 3 3 4 3 1 3 1 4 1 4 1 3 0 0 1 4 1 1 4 1 1
[1259] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
[1296] 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 0 1 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 0 0 0 0
[1333] 0 0 1 0 1 0 0 0 0 0 1 3 3 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3
[1370] 3 3 3 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
[1407] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 3 4 4 1 4 4 4 4 4 4 4 4 4
[1444] 0 0 3 0 3 4 1 1 4 4 4 4 4 3 4 1 4 0 0 3 1 0 4 0 4 0 4 4 0 4 0 1 4 4 0 4 4
[1481] 4 4 1 4 4 4 4 4 1 4 4 4 1 1 1 4 3 4 4 4 4 0 0 3 0 4 4 1 4 0 4 0 4 0 4 1 0
[1518] 1 1 4 4 4 4 0 0 0 4 4 1 1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[1555] 1 1 1 1 1 1 1 1 1 1 2 1 1 0 1 1 4 1 0 2 2 2 2 1 1 1 2 0 1 1 1 1 1 1 0 4 1
[1592] 2 1 2 1 0 0 1 4 1 4 4 4 4 4 0 4 4 4 3 3 4 4 3 4 4 4 4 4 3 3 4 4 1 4 4 4 0
[1629] 4 2 1 0 4 2 4 4 3 2 4 4 3 3 4 3 4 2 1 2 4 0 3 4 4 4 0 0 2 4 4 0 0 0 4 4 4
[1666] 0 0 0 0 4 4 4 4 1 4 1 4 4 4 4 4 4 0 0 2 4 2 3 2 4 2 0 4 4 0 4 0 1 4 1 4 0
[1703] 4 2 4 2 4 4 1 4 4 4 4 2 4 4 4 4 3 1 2 4 4 0 3 2 3 4 1 4 4 4 4 4 3 4 4 4 0
[1740] 3 4 4 4 4 2 4 4 3 3 4 0 4 0 4 1 4 2 4 4 4 3 4 3 2 4 4 4 4 0 4 4 0 4 0 0 4
[1777] 0 0 4 4 0 4 4 2 0 0 0 0 4 4 4 0 4 2 2 2 4 4 4 0 0 3 4 0 0 4 4 0 4 4 4 4 1
[1814] 1 4 1 1 4 0 4 4 4 4 4 4 4 4

In [44]:
lgbregmin

[1]  3.564855920  3.463392005  2.637323079  3.488242477  2.331450997
   [6]  3.446735543  2.154213931  2.571876247  2.503234870  2.115808564
  [11]  2.235047645  2.631426715  1.727410593  3.762152210  3.076999609
  [16]  0.442942880  1.320222841  1.846140760  3.917723320  1.133443116
  [21]  4.020523565  3.597272405  3.692011830  3.684839110  3.630206795
  [26]  3.813775496  3.631481435  2.237320164  3.464498362  2.851584081
  [31]  0.879943014  1.058362492  1.611592816  2.562237361  2.558963463
  [36]  2.136600732  2.101003014  3.264376667  1.888989250  3.098428769
  [41]  3.537893099  2.270648246  2.683221669  1.829434295  1.270130658
  [46]  2.364457119  1.751647996  3.635651583  3.204163657  3.459672484
  [51]  3.222582090  1.886638673  0.952240348  3.621298512  1.630296215
  [56]  3.047927174  3.231756392  3.264498771  3.387701611  3.057442388
  [61]  1.468490715  3.414473722  2.435274401  0.712763418  3.031008165
  [66]  3.081337143  3.265355404  2.760498072  1.115614237  3.422431411
  [71]  2.064369761  3.356686812  3.750278623  2.184168642  2.816774263
  [76]  3.577582247  2.664635364  2.621976792  3.020671730  3.536105026
  [81]  3.166343976  3.237664923  2.291588363  0.964842224  1.568040594
  [86]  3.291027513  0.734485684  0.169911264 -0.083259430  0.666579980
  [91]  0.864216759  0.205370919  0.029367682 -0.029232633  0.370209635
  [96] -0.342266534  0.004891124  3.269672848  2.625137631  2.552916122
 [101]  3.503426807  3.489839463  3.123154865  3.439794942  3.289295792
 [106]  0.231030956  0.545673038  3.321613709  3.424037396  3.393037982
 [111]  2.870167983  3.071367995  3.223458849  2.438066803  2.533084422
 [116]  2.226052735  2.023092025  0.344021671  0.545937492  1.912703246
 [121]  1.119249651  1.553982482  2.620415666  1.977691383  0.380150471
 [126]  2.511192069  2.501231326  2.243769501  3.633366814  1.947394194
 [131]  2.879361937  3.333673630  3.217340758  1.809385196  2.788788009
 [136]  3.201100029  3.381758528  3.232496369  1.075587735  2.036551351
 [141]  3.759707031  3.548538078  3.620677209  1.819687453  3.425134016
 [146]  0.648171793  1.235453501  4.032772691  2.827844043  2.005553076
 [151]  1.816227958  1.271864392  1.924031811  3.517176608  2.565245646
 [156]  3.886416770  3.872769572  0.751898087  3.312965999  1.722521581
 [161]  2.482072640  3.982563844  1.608525644  0.959229746  1.330429976
 [166]  3.538987679  3.972788109  2.586474008  2.610991591  1.463980398
 [171]  1.136814558  1.000087715  1.018467877  1.749889618  2.685074755
 [176]  3.702695703  3.877952404  3.843300502  0.262300227  0.745254276
 [181]  0.507999566  0.976256030  3.081661811  3.344493441  3.420677355
 [186]  2.916388822  3.563672845  3.448943054  3.429013572  2.740771383
 [191]  3.798225932  3.333569547  1.574692879  3.778893383  3.862952041
 [196]  3.742001909  0.776501769  1.523437893  1.755850786  1.438819247
 [201]  4.038705143  2.542717705  2.740528431  2.509916332  1.862304629
 [206]  2.366925421  2.803438812  3.577324371  3.935868778  3.612793046
 [211]  3.947377203  2.243708497  2.930498245  1.548436562  3.349096614
 [216]  3.748734244  0.957479512  2.116605878  2.569922557  2.714982533
 [221]  0.653417869  2.390381063  3.455137595  3.416859457  3.810349992
 [226]  0.396904077  0.656650818  0.620031508  1.306057902  3.111979943
 [231]  3.477272457  3.418251094  2.247964066  1.263158837  2.682094544
 [236]  2.665858307  2.470537711  3.094751919  2.376757769  1.805626318
 [241]  1.382053273  3.769938080  2.523944749  3.137530953  3.528977122
 [246]  3.364645556  3.100741654  2.894941218  0.785102501  0.352827424
 [251]  1.179059857  2.107874447  1.988026073  2.098077403  1.842407228
 [256]  2.254960636  2.366638168  3.366559148  1.707765937  2.808766470
 [261]  1.933724414  2.419147098  1.264414114  1.589386820  2.118891564
 [266]  2.025007466  1.493410580  2.892621135  2.029396123  3.010977472
 [271]  3.737290535  2.917922603  3.058698425  1.633730048  2.007740497
 [276]  2.542528638  1.920845856  3.6529499

In [45]:
lgbclsmin

0.002942123,0.0151650610,0.026998840,0.075733975,0.87916000
0.006448491,0.0192777623,0.046485864,0.146486670,0.78130121
0.113042770,0.0187948466,0.061375639,0.022287168,0.78449958
0.125768344,0.0277627879,0.074662872,0.019428635,0.75237736
0.003709867,0.0071594854,0.889385754,0.012172559,0.08757233
0.003528281,0.0189394788,0.013318147,0.124429313,0.83978478
0.125232891,0.0554033239,0.112486445,0.113909326,0.59296801
0.030154047,0.0576170287,0.014530809,0.835396741,0.06230137
0.045979937,0.0583694694,0.070279951,0.761603253,0.06376739
0.078336025,0.0163406165,0.751616429,0.032320150,0.12138678
0.009412237,0.7013124003,0.039437138,0.168033331,0.08180489


In [46]:
datalength=dim(data)[1]
lgbclsm=matrix(lgbclsmin,m,datalength)
lgbclsminr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsminr[i,]=which.max(lgbclsm[,i])
}
lgbclsminr=lgbclsminr-1

In [47]:
lgbclsminr

3
1
4
0
2
2
1
0
4
1
4


In [48]:
xgbregmean=predict(xgb_mean_reg,alldataxgb)
xgbclsmean=predict(xgb_mean_cl,alldataxgb)
lgbregmean=predict(lgb_mean_reg,as.matrix(data))
lgbclsmean=predict(lgb_mean_cl,as.matrix(data))

In [49]:
lgbclsm=matrix(lgbclsmean,m,datalength)
lgbclsmeanr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsmeanr[i,]=which.max(lgbclsm[,i])
}
lgbclsmeanr=lgbclsmeanr-1

In [50]:
preallmin=cbind(xgbclsmin,xgbregmin)
preallmin=cbind(preallmin,lgbclsminr)
preallmin=cbind(preallmin,lgbregmin)

In [51]:
colnames(preallmin)=c('xgbclsmin','xgbregmin','lgbclsmin','lgbregmin')
head(preallmin)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
4,3.902582,3,3.564856
4,3.735962,1,3.463392
4,3.824364,4,2.637323
4,3.895803,0,3.488242
2,2.067485,2,2.331451
4,3.960983,2,3.446736


In [52]:
preallmean=cbind(xgbclsmean,xgbregmean)
preallmean=cbind(preallmean,lgbclsmeanr)
preallmean=cbind(preallmean,lgbregmean)

In [53]:
head(preallmean)

xgbclsmean,xgbregmean,,lgbregmean
3,3.0057189,4,3.0898321
4,3.9117641,1,3.2540208
4,3.9151545,0,2.5287291
3,2.9736011,0,2.3288804
0,0.1768687,2,0.9346416
3,2.9403007,4,3.0254235


In [54]:
colnames(preallmean)=c('xgbclsmean','xgbregmean','lgbclsmean','lgbregmean')
head(preallmean)

xgbclsmean,xgbregmean,lgbclsmean,lgbregmean
3,3.0057189,4,3.0898321
4,3.9117641,1,3.2540208
4,3.9151545,0,2.5287291
3,2.9736011,0,2.3288804
0,0.1768687,2,0.9346416
3,2.9403007,4,3.0254235


In [55]:
end_time = Sys.time()
time_matrix[4,]=end_time-start_time

In [56]:
result_list=list(preallmin,preallmean,time_matrix)

In [58]:
save(result_list, file = "Daily_ets_opt_pre_result.RData")

In [57]:
time_matrix

user_time,system_time,elapsed_time
3.624090,3.624090,3.624090
10.518797,10.518797,10.518797
17.256910,17.256910,17.256910
1.565955,1.565955,1.565955


In [31]:
importance_matrix1 <- xgb.importance(model = xgb_mean_cl)
importance_matrix2 <- xgb.importance(model = xgb_mean_reg)
importance_matrix3 <- lgb.importance(lgb_mean_cl)
importance_matrix4 <- lgb.importance(lgb_mean_reg)

importance_matrix1=importance_matrix1[order(importance_matrix1$Feature),]
importance_matrix2=importance_matrix2[order(importance_matrix2$Feature),]
importance_matrix3=importance_matrix3[order(importance_matrix3$Feature),]
importance_matrix4=importance_matrix4[order(importance_matrix4$Feature),]

importance_matrix_mean=(importance_matrix1[,2:4]+importance_matrix2[,2:4]+importance_matrix3[,2:4]+importance_matrix4[,2:4])/4

importance=cbind(importance_matrix1[,1],importance_matrix_mean)

write.csv(importance,'d_ets_imp_mean.csv')